In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from scipy.signal import medfilt
import pickle as pk

# LoadData

In [2]:
df = pd.read_csv('train.csv', sep=';')

In [3]:
A = np.load('penalty_matrix.npy')
def score(y_true, y_pred, **kwargs):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

def Normalize_Data_by_Well(dataFrame,col='GR'):
    wells = dataFrame['WELL'].unique()
    values = []
    for well in wells:
        min_value = dataFrame[dataFrame['WELL'] == well][col].min()
        max_value = dataFrame[dataFrame['WELL'] == well][col].max()
        col_normalized = (dataFrame[dataFrame['WELL'] == well][col].values-min_value)/(max_value-min_value)
        values = values + list(col_normalized)
    return values

def Delta_Feature(dataFrame,col='GR',inverted=False):
    wells = dataFrame['WELL'].unique()
    values = []
    for well in wells:
        col_values = dataFrame[dataFrame['WELL'] == well][col].values
        col_values_ = np.array([col_values[0]]+list(col_values[:-1]))
        delta_col_values = col_values-col_values_
        if inverted:
            delta_col_values=-delta_col_values
        values = values + list(delta_col_values)
    return values


def Add_New_Features(dataFrame):
    data = dataFrame['RHOB'].values  
    data = ((154.497/data) - 57.261)
    dataFrame['Carbon_Index'] = data
    dataFrame['Normalized_RHOB'] = Normalize_Data_by_Well(dataFrame,col='RHOB')
    dataFrame['Normalized_GR'] = Normalize_Data_by_Well(dataFrame,col='GR')    
    dataFrame['Delta_DTC'] = Delta_Feature(dataFrame,col='DTC',inverted=True)
    dataFrame['Delta_RHOB'] = Delta_Feature(dataFrame,col='RHOB',inverted=True)    
    dataFrame['Delta_GR'] = Delta_Feature(dataFrame,col='GR',inverted=True)
    dataFrame['Delta_DEPTH_MD'] = Delta_Feature(dataFrame,col='DEPTH_MD')
    dataFrame['Delta_Carbon_Index'] = Delta_Feature(dataFrame,col='Carbon_Index')
    
    return dataFrame

def Fill_Data(dataFrame,fill_formation,fill_BS,fill_with_median):
    dataFrame.FORMATION = dataFrame.FORMATION.fillna(fill_formation)
    dataFrame.BS = dataFrame.BS.fillna(fill_BS)
    dataFrame.fillna(fill_with_median, inplace=True)
    dataFrame = pd.get_dummies(dataFrame, columns=['FORMATION'], drop_first=True)
    return dataFrame

# Add Features

In [4]:
df = Add_New_Features(df)

# Selected Features

In [5]:
df = df[['DEPTH_MD', 'FORMATION', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB','GR',
         'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'DRHO','FORCE_2020_LITHOFACIES_LITHOLOGY',
         'Carbon_Index','Delta_Carbon_Index','Delta_DTC','Delta_RHOB','Delta_DEPTH_MD',
         'Delta_GR','Normalized_GR','Normalized_RHOB','X_LOC','Y_LOC']]
df.head()

,DEPTH_MD,FORMATION,CALI,RSHA,RMED,RDEP,RHOB,GR,NPHI,PEF,...,Carbon_Index,Delta_Carbon_Index,Delta_DTC,Delta_RHOB,Delta_DEPTH_MD,Delta_GR,Normalized_GR,Normalized_RHOB,X_LOC,Y_LOC
0,494.528,NaN,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,NaN,20.915468,...,24.735691,0.000000,-0.000000,-0.000000,0.000,-0.000000,0.150172,0.314847,437641.96875,6470972.5
1,494.680,NaN,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,NaN,19.383013,...,24.492376,-0.243315,0.527710,-0.005608,0.152,0.937965,0.148269,0.318528,437641.96875,6470972.5
2,494.832,NaN,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,NaN,22.591518,...,24.202299,-0.290077,0.429855,-0.006729,0.152,4.440887,0.139258,0.322946,437641.96875,6470972.5
3,494.984,NaN,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,NaN,32.191910,...,24.400797,0.198498,0.024185,0.004610,0.152,1.943077,0.135315,0.319919,437641.96875,6470972.5
4,495.136,NaN,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,NaN,38.495632,...,24.916765,0.515968,0.021088,0.011879,0.152,1.149780,0.132982,0.312121,437641.96875,6470972.5


# Fill Data

In [6]:
fill_formation = df.FORMATION.value_counts().index[0]
fill_BS = 12.250001
fill_with_median = df.median()


In [7]:
df = Fill_Data(df,fill_formation,fill_BS,fill_with_median)
df.head()

,DEPTH_MD,CALI,RSHA,RMED,RDEP,RHOB,GR,NPHI,PEF,DTC,...,FORMATION_Tarbert Fm.,FORMATION_Tau Fm.,FORMATION_Tor Fm.,FORMATION_Tryggvason Fm.,FORMATION_Tuxen Fm.,FORMATION_Ty Fm.,FORMATION_Ty Mb.,FORMATION_Ula Fm.,FORMATION_Utsira Fm.,FORMATION_Vaale Fm.
0,494.528,19.480835,1.39902,1.611410,1.798681,1.884186,80.200851,0.3268,20.915468,161.131180,...,0,0,0,0,0,0,0,0,1,0
1,494.680,19.468800,1.39902,1.618070,1.795641,1.889794,79.262886,0.3268,19.383013,160.603470,...,0,0,0,0,0,0,0,0,1,0
2,494.832,19.468800,1.39902,1.626459,1.800733,1.896523,74.821999,0.3268,22.591518,160.173615,...,0,0,0,0,0,0,0,0,1,0
3,494.984,19.459282,1.39902,1.621594,1.801517,1.891913,72.878922,0.3268,32.191910,160.149429,...,0,0,0,0,0,0,0,0,1,0
4,495.136,19.453100,1.39902,1.602679,1.795299,1.880034,71.729141,0.3268,38.495632,160.128342,...,0,0,0,0,0,0,0,0,1,0


# Train 

In [8]:
output = "FORCE_2020_LITHOFACIES_LITHOLOGY"

X = df.drop([output], axis=1)
y = df[output]

lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
y = y.map(lithology_numbers)

In [9]:
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
columns = X.columns

# Submission

In [10]:
df_test = pd.read_csv('test.csv', sep=';')
df_test = Add_New_Features(df_test)
df_test = Fill_Data(df_test,fill_formation,fill_BS,fill_with_median)


In [11]:
df_test_aux = pd.DataFrame(columns=columns, data=np.zeros((df_test.shape[0], X.shape[1])))
for col in df_test_aux.columns:
    if col in df_test.columns:
        df_test_aux[col] = df_test[col]

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
x = X_train.copy()
y = y_train.copy()

n_splits=5
folds = StratifiedKFold(n_splits, random_state=42, shuffle=True)
scores = []
features = x.columns
feature_importance_df = pd.DataFrame()
yt = np.zeros((df_test_aux.shape[0],12))
for i, (trn_idx, val_idx) in enumerate(folds.split(x.values, y.values)):    
    model = RandomForestClassifier(n_estimators=500, random_state= 42, min_samples_split=13,
                                 class_weight='balanced', n_jobs=-1,criterion = 'entropy')
    model.fit(x.iloc[trn_idx], y.iloc[trn_idx])
    yt += model.predict_proba(df_test_aux)
    y_pred_test = model.predict(X_test)
    print(score(y_test.values, y_pred_test))
    pk.dump(model, open('trained_model_'+str(i+1)+'.pkl', 'wb'))

In [12]:
fill_information= [fill_formation,fill_BS,fill_with_median,columns]
pk.dump(fill_information, open('fill_information.pkl', 'wb'))

In [13]:
class Model(object):
    def __init__(self, model_files, fill_information='fill_information.pkl'):
        # Load pre-trained models from file
        self.model_files=model_files
        
        #self.model=[]
        #for i,file in enumerate(model_files):
        #    self.model.append(pk.load(open(file, 'rb')))
        
        # Load a "filles-information" from file
        self.fill_information = pk.load(open(fill_information, 'rb'))
        
    def _preprocess(self, features):
        # Method to be run before inference.
        features = Add_New_Features(features)
        features = Fill_Data(features,self.fill_information[0],self.fill_information[1],self.fill_information[2])
        complete_features = pd.DataFrame(columns=self.fill_information[-1], 
                                   data=np.zeros((features.shape[0], len(self.fill_information[-1]))))
        for col in complete_features.columns:
            if col in features.columns:
                complete_features[col] = features[col]
        
        return complete_features
    def corrections(self,features):
        for well in features['WELL'].unique():
            values_by_facies = features[features['WELL']==well]['Prediction'].value_counts()
            values_by_facies = values_by_facies[values_by_facies < 20].index
            for value in values_by_facies :
                v = features[(features['WELL']==well)]['Prediction'].values
                v[v==value] = features[features['WELL']==well]['Prediction'].value_counts().index[0].astype('int64')
                features.loc[features['WELL']==well,'Prediction'] = v
            
            features.loc[features['WELL']==well,'Prediction'] = medfilt(features[features['WELL']==well]['Prediction'].values,
                                                                        kernel_size=3)
        return features['Prediction'].values
    def predict(self, features):
        # This function should be able to take in features in their
        # raw, unprocessed form as read from the file test.csv and
        # return predictions as an array integers of the same length        
        features_base = features.copy()
        X = self._preprocess(features)
        y_t = np.zeros((X.shape[0], 12))
        for model_file in self.model_files:
            model = pk.load(open(model_file, 'rb'))        
            y_t += model.predict_proba(X)
            y_pred_test = model.predict(X_test)
            print(score(y_test.values, y_pred_test))            
            del model # delete object to clear memory
        predictions = np.argmax(y_t, axis=-1)
        category_to_lithology = {y:x for x,y in lithology_numbers.items()}
        test_prediction_for_submission = np.vectorize(category_to_lithology.get)(predictions)

        features_base['Prediction'] = test_prediction_for_submission        
        #return predictions       
        return self.corrections(features_base)

In [14]:
open_test_features = pd.read_csv('test.csv', sep=';')

In [22]:
model = Model(['trained_model_1.pkl','trained_model_2.pkl',
               'trained_model_3.pkl','trained_model_4.pkl',
               'trained_model_5.pkl'], 'fill_information.pkl')

In [23]:
test_prediction = model.predict(open_test_features)

-0.15847564960722418
-0.1583469669333584
-0.15848312494927447
-0.15784451715697792
-0.1589594110284789


In [24]:
np.savetxt('test_predictions.csv', test_prediction, header='lithology', comments='', fmt='%i')

In [25]:
!pip freeze > requirements.txt